# Responsible AI System for Credit Score Prediction

A responsible AI system for credit score prediction involves several key principles:

## 1. Fairness & Bias Mitigation
- **Check for Bias in Data**: Analyze whether certain demographic groups (e.g., gender, marital status, income levels) are disproportionately affected.  
- **Fair Representation**: Use techniques like re-sampling, re-weighting, or adversarial debiasing to balance the dataset.  
- **Fair Model Evaluation**: Compare accuracy across different demographic subgroups to ensure fairness.  

## 2. Explainability & Transparency
- **Use Explainable Models**: While Random Forest and XGBoost are powerful, tools like SHAP (SHapley Additive Explanations) can help interpret feature contributions.  
- **Provide Clear Justifications**: Show why a customer received a specific credit score prediction.  

## 3. Privacy & Security
- **Anonymize Data**: Avoid using personally identifiable information (PII) like names or addresses.  
- **Secure Model Deployment**: Encrypt user inputs and outputs.  
- **Comply with Regulations**: Adhere to laws like GDPR or CCPA to protect user privacy.  

## 4. Robustness & Reliability
- **Detect Data Drift**: Monitor changes in customer behavior over time to prevent outdated models from making incorrect predictions.  
- **Handle Edge Cases**: Implement safeguards to manage uncertain or extreme cases in credit scoring.  

## 5. Ethical AI Decision-Making
- **Allow for Human Oversight**: Ensure that financial experts can review automated decisions when necessary.  
- **Transparent User Communication**: Provide customers with reasons behind their credit score and actionable insights to improve it.  
 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from aif360.sklearn.metrics import disparate_impact_ratio, statistical_parity_difference
# Apply Bias Detection Metrics
from fairlearn.metrics import demographic_parity_difference, selection_rate

# Data Preprocessing

In [3]:
# Load dataset
df = pd.read_csv("BankChurners.csv")

In [ ]:
df.head()

In [5]:
# Drop unnecessary columns (Client ID and Naive Bayes columns)
df.drop(columns=['CLIENTNUM', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 
                'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], inplace=True)

# Anonymize Sensitive Data
df.drop(columns=['Customer_Age', 'Marital_Status'], inplace=True)  # Remove personally identifiable attributes


# Feature Extraction

In [6]:
# Encode categorical features
label_encoders = {}
for col in ['Attrition_Flag', 'Gender', 'Education_Level', 'Income_Category', 'Card_Category']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [7]:
# Define "Credit Score" Proxy: Categorizing Credit Score
score_bins = [0, 5000, 10000, 15000, np.inf]
score_labels = ['Poor', 'Fair', 'Good', 'Excellent']
df['Credit_Score_Category'] = pd.cut(df['Credit_Limit'] * (1 - df['Avg_Utilization_Ratio']), bins=score_bins, labels=score_labels)
le_class = LabelEncoder()
df['Credit_Score_Category'] = le_class.fit_transform(df['Credit_Score_Category'])

# Random Forest Classifier

In [8]:
# Define Features and Target for Classification
X = df.drop(columns=['Credit_Score_Category'])
y_class = df['Credit_Score_Category']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=0.2, random_state=42)


In [ ]:
X.head()

In [ ]:
# Standardizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
X_test.head()

In [12]:
# Train Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Bias Detection Metrics

In [15]:
# Convert X_test_scaled (NumPy array) back to DataFrame with original column names
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

# Predict using the RandomForestClassifier
y_pred = rf_clf.predict(X_test_scaled_df)


In [13]:
# Extract Gender Before Encoding
protected_attribute = df.loc[X_test.index, 'Gender']  # Fetch original values

In [ ]:
from `fairlearn` is a Python package that provides tools for assessing and mitigating unfairness in machine learning models. In this code snippet, the `fairlearn.metrics` module is being used to calculate the selection rates for male and female groups based on the predictions made by a random forest classifier (`rf_clf`). The selection rates are then used to compute the gender disparate impact ratio, which measures the difference in selection rates between male and female groups. The goal is to identify and address any potential biases or disparities in the model's predictions based on the protected attribute (Gender) in a fair and transparent manner.
fairlearn.metrics import selection_rate
import numpy as np

# Convert Gender back to categorical values if needed
protected_attribute = df.loc[X_test.index, 'Gender']  # Fetch original values before encoding
protected_attribute = protected_attribute.map({0: "Male", 1: "Female"})  # Ensure correct mapping

# Convert X_test_scaled back to DataFrame for correct indexing
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

# Get Predictions
y_pred = rf_clf.predict(X_test_scaled_df)

# Ensure `protected_attribute` and `y_pred` are aligned
protected_attribute = protected_attribute.loc[X_test_scaled_df.index]

# Compute selection rates for Male & Female (avoid division by zero)
male_selection_rate = selection_rate(y_test[protected_attribute == "Male"], y_pred[protected_attribute == "Male"])
female_selection_rate = selection_rate(y_test[protected_attribute == "Female"], y_pred[protected_attribute == "Female"])

# Avoid NaN by handling zero selection rates
if male_selection_rate == 0 or female_selection_rate == 0:
    gender_disparate_impact = np.nan  # Cannot compute ratio if one group has zero selection
    print("Warning: One of the gender groups has zero selection rate.")
else:
    gender_disparate_impact = female_selection_rate / male_selection_rate

print("Male Selection Rate:", male_selection_rate)
print("Female Selection Rate:", female_selection_rate)
print("Gender Disparate Impact Ratio:", gender_disparate_impact)


Male Selection Rate: 0.0
Female Selection Rate: 0.0
Gender Disparate Impact Ratio: nan


In [ ]:
if protected_attribute.dtype != 'O':  # If it's numerical
    protected_attribute = protected_attribute.map({0: "Male", 1: "Female"})
    print("in")


in


In [ ]:

# Extract Gender Before Encoding
protected_attribute = df.loc[X_test.index, 'Gender']  # Fetch original values

# Ensure Gender is categorical (map numbers back to labels if needed)
if protected_attribute.dtype != 'O':  # If it's numerical
    protected_attribute = protected_attribute.map({0: "Male", 1: "Female"})

# Fairness Metrics Calculation
try:
    gender_disparate_impact = disparate_impact_ratio(y_test, rf_clf.predict(X_test_scaled), sensitive_features=protected_attribute)
    gender_statistical_parity = demographic_parity_difference(y_test, rf_clf.predict(X_test_scaled), sensitive_features=protected_attribute)
except TypeError:  # Handle old fairlearn versions
    gender_disparate_impact = disparate_impact_ratio(y_test, rf_clf.predict(X_test_scaled), prot_attr=protected_attribute)
    gender_statistical_parity = demographic_parity_difference(y_test, rf_clf.predict(X_test_scaled), prot_attr=protected_attribute)

print("Gender Disparate Impact Ratio:", gender_disparate_impact)
print("Gender Statistical Parity Difference:", gender_statistical_parity)


# Explainability with SHAP

In [ ]:
explainer = shap.TreeExplainer(rf_clf)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)


# Predict Credit Score of a New Customer

In [ ]:
def predict_credit_score(new_customer):
    new_customer_df = pd.DataFrame([new_customer])
    
    # Ensure all required columns exist and reorder columns to match training data
    for col in label_encoders:
        if col in new_customer_df.columns:
            new_customer_df[col] = label_encoders[col].transform(new_customer_df[col])
    
    # Reindex the dataframe to ensure all necessary features are included
    new_customer_df = new_customer_df.reindex(columns=X.columns, fill_value=0)
    
    # Standardize features
    new_customer_scaled = scaler.transform(new_customer_df)
    
    # Predict credit score category
    predicted_score_category = rf_clf.predict(new_customer_scaled)[0]
    explanation = explainer.shap_values(new_customer_scaled)
    shap.force_plot(explainer.expected_value[predicted_score_category], explanation[predicted_score_category], new_customer_scaled)
    
    return le_class.inverse_transform([predicted_score_category])[0]

# Example Usage

In [ ]:
customers_data = [
    {
        "Gender": "M",
        "Dependent_count": 3,
        "Education_Level": "Graduate",
        "Income_Category": "$60K - $80K",
        "Card_Category": "Blue",
        "Months_on_book": 39,
        "Total_Relationship_Count": 5,
        "Months_Inactive_12_mon": 2,
        "Contacts_Count_12_mon": 3,
        "Credit_Limit": 12691.0,
        "Total_Revolving_Bal": 777,
        "Avg_Open_To_Buy": 11914.0,
        "Total_Amt_Chng_Q4_Q1": 1.335,
        "Total_Trans_Amt": 1144,
        "Total_Trans_Ct": 42,
        "Total_Ct_Chng_Q4_Q1": 1.625,
        "Avg_Utilization_Ratio": 0.061,
        "Attrition_Flag": "Existing Customer"
    }
]

for customer in customers_data:
    predicted_score_category = predict_credit_score(customer)
    print("Predicted Credit Score Category:", predicted_score_category)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


IndexError: index 3 is out of bounds for axis 0 with size 1